In [1]:
from Detector.input_producer import InputProducer
from Detector.RetinaNet import RetinaNet
from utils.preprocess import *
import collections
import tensorflow as tf
import os
import numpy as np
from utils.bbox import change_box_order
os.environ["CUDA_VISIBLE_DEVICES"]=""

print("1")
slim = tf.contrib.slim

FLAGS = tf.app.flags.FLAGS

tf.logging.set_verbosity(tf.logging.WARN)
tf.app.flags.DEFINE_string('f', '', 'kernel')
#### Input pipeline
tf.app.flags.DEFINE_integer('input_size', 608,
                            """Input size""")
tf.app.flags.DEFINE_integer('num_classes', 20,
                            """number of classes""")
tf.app.flags.DEFINE_float('cls_thresh', 0.5,
                            """thresh for class""")
tf.app.flags.DEFINE_float('nms_thresh', 0.5,
                            """thresh for nms""")
tf.app.flags.DEFINE_integer('max_detect', 300,
                            """num of max detect (using in nms)""")


1


In [3]:

net = RetinaNet("resnet50")

input_features = []
print("2")
InputFeatures = collections.namedtuple('InputFeatures', ('image', 'loc', 'cls'))
input_producer = InputProducer()

split_name = 'train_2000'
print("3")
dataset = input_producer.get_split(split_name, '/root/DB/VOC/VOC2012/tfrecord/')
print("4")
provider = slim.dataset_data_provider.DatasetDataProvider(
                dataset,
                num_readers=4,
                common_queue_capacity=80,
                common_queue_min=40,
                shuffle=True)
print("5")
_images, _bboxes, _labels = provider.get(['image', 'object/bbox', 'object/label'])

crop_img, crop_box = random_crop(_images, _bboxes)
'''
_images, _bboxes = net.preprocess_image(_images, _bboxes)

_bboxes, _labels = self.encode(_bboxes, _labels)

total_anchor = net._get_anchor_boxes()
total_anchor = change_box_order(total_anchor, "xywh2xyxy")
'''

2
3
4
5


TypeError: Tensor objects are not iterable when eager execution is not enabled. To iterate over this tensor use tf.map_fn.

In [ ]:
import cv2
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt

VOC = {1 : "motorbike", 2 : "car", 3 : "person", 4 : "bus", 5 : "bird", 6 : "horse", 7 : "bicycle", 8 : "chair", 9 : "aeroplane", 10 : "diningtable", 11 : "pottedplant", 12 : "cat", 13 : "dog", 14 : "boat", 15 : "sheep", 16 : "sofa", 17 : "cow", 18 : "bottle", 19 : "tvmonitor", 20 : "train"}

def draw_boxes(img, bboxes, classes):
    if len(bboxes) == 0:
        return img

    height, width, _ = img.shape
    image = Image.fromarray(img)
    font = ImageFont.truetype(
        font='/root/FiraMono-Medium.otf',
        size=np.floor(3e-2 * image.size[1] + 0.4).astype('int32'))

    thickness = (image.size[0] + image.size[1]) // 300
    draw = ImageDraw.Draw(image)

    for box, category in zip(bboxes, classes):
        y1, x1, y2, x2 = [int(i) for i in box]
        #x1, y1, x2, y2 = [int(i) for i in box]
        
        p1 = (x1, y1)
        p2 = (x2, y2)

        label = '{} '.format(category.title())
        label_size = draw.textsize(label)
        text_origin = np.array([p1[0], p1[1] - label_size[1]])

        color = np.array([0, 255, 0])
        for i in range(thickness):
            draw.rectangle(
                [p1[0] + i, p1[1] + i, p2[0] - i, p2[1] - i],
                outline=tuple(color))

        draw.rectangle(
            [tuple(text_origin),
             tuple(text_origin + label_size)],
            fill=tuple(color))

        draw.text(
            tuple(text_origin),
            label, fill=(0, 0, 0),
            font=font)

    del draw
    return np.array(image)

In [ ]:
def expand_dimension(input):
    out = []
    for i in input:
        out.append(np.expand_dims(i, 0))
    return out

In [ ]:
init_op = tf.group(tf.global_variables_initializer(),
                           tf.local_variables_initializer())
from matplotlib.pyplot import figure

In [ ]:
with tf.Session() as sess:
    with slim.queues.QueueRunners(sess):
        for i in range(10):
            #np_image, np_bbox, np_label = sess.run([image, bboxes, labels])
            #np_loc, np_cls = sess.run([loc, cls])
            #np_anchor = sess.run(total_anchor)
            
            #np_image, np_bbox, np_label = sess.run([image2, boxes2, _labels])
            #np_image, np_bbox, np_label = expand_dimension([np_image, np_bbox, np_label])
            
            np_image, np_bbox, np_label = sess.run([crop_img, crop_box, _labels])
            
            #h, w, _ = np_image.shape
            
            #np_image = np.array(np_image, np.uint8)
            
            np_class = [VOC[l] for l in np_label[0]]
            
            figure(figsize = (12,12))
            print(np_image[0].shape, np_bbox[0], np_class)
            img = cv2.resize(np_image[0], (608, 608))
            img = draw_boxes(img, np_bbox[0]*608, np_class)
            plt.imshow(img)
            plt.show()
            
            
            continue
            
            np_label = [VOC[idx] for idx in np_label]
            print(np_image.shape, np_label)
            
            #np_bbox = (np_bbox * [w, h, w, h]) #.astype(np.int32)
            
            '''
            for i, box in enumerate(np_anchor):
                box = (box / 608) * 1200
                print(i+1, box)
                s_img = img.copy()
                s_img = cv2.rectangle(s_img, (box[0], box[1]), (box[2], box[3]), (0,255,0), 2)
            
                figure(figsize = (12,12))
                plt.imshow(s_img)
                plt.show()
                if i == 30:
                    break
            '''

            img = draw_boxes(np_image, np_bbox, np_label)
            figure(figsize = (12,12))
            plt.imshow(img)
            plt.show()
